NoteBook to try some example algorithms in python from the reinforcement learning book. Originally the algorithms I implemented on MATLAB have been translated into python code. 
The basis problem is a grid of squares where the terminal states are square 1,and the final square. The policy implemented contains a grid of numbers, where in each square is the index the player should move to from that square.


# The Actual Value Function (for the policy pi)

In [2]:
pi = np.array([0,0,1,0,1,8,7,8,8]) #policy array - each number points to the next location

P = np.array([[1,0,0,0,0,0,0,0,0],
             [1,0,0,0,0,0,0,0,0],
             [0,1,0,0,0,0,0,0,0],
             [1,0,0,0,0,0,0,0,0],
             [0,1,0,0,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,1],
             [0,0,0,0,0,0,0,1,0],
             [0,0,0,0,0,0,0,0,1],
             [0,0,0,0,0,0,0,0,1]]) #Prob matrix of policy

VActual = np.matmul(np.linalg.inv(np.eye(9)-gamma*P),R) #Solve bellman equation
VActual

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 16 is different from 9)

# TD[0] (One Step TD) Implementation

In [ ]:
import numpy as np

alpha, gamma, n = 0.5, 0.9, 200    #step parameter, discount factor, number of episodes
V = np.zeros((9,1))                #arbitrary initial V estimate  
R = np.ones((9,1))                    
R[0], R[8] = 0, 0                  #reward array
S0_arr = np.random.randint(0,9,n)  #Generate starting states (1,to 9)

#initially use for loops though I'm aware it's not the fastest way.
for i in range(n):
    S = S0_arr[i]          #choose random starting state
    running = True         #true until the terminal state is reached
    while running:
        if S == 0 or S == 8:
            running = False
        else:
            Snew = pi[S]
            V[S] = V[S] + alpha*(R[S]+gamma*V[Snew]-V[S])
            S = Snew

print("The difference between the estimate and the actual value function is : \n", V-VActual)


# Simple 3x3 Grid Q Learning Implementation

Slightly different implementation of the 3x3 grid idea, where the Q table has actions 0-3; up, right, down, left in that order ascending. Rows of the Q table are the states and the columns are the possible actions. This method learns the optimum policy as the iterations of episodes update the Q values.

In [3]:
import numpy as np

#Takes the arr from Q with one state and multiple actions, finds highest value and returns the index of the action column
def get_max_A(arr):
    indices = np.where(arr == np.amax(arr)) #indices of maximum Q value of the array
    return indices[0][0]

#takes input of action and returns new state according to board physics
def change_state(S,A):
    #Don't move if at the board edge. Otherwise 0 up, 1 right, 2 down, 3 left
    if((S % 3 == 0 and A == 3) or (S < 3 and A == 0) or (S > 5 and A == 2) or (S % 3 == 2 and A == 1)):
        return S
    elif(A == 0):
        return S-3
    elif(A == 1):
        return S+1
    elif(A == 2):
        return S+3
    elif(A == 3):
        return S-1
    else:
        print('change_state() broken at state {} and action {}'.format(S,A))
        return S

alpha, gamma, n = 0.5, 0.9, 200    #step parameter, discount factor, number of episodes

Q = np.zeros((9,4))                #arbitrary initial Q Table estimate ((9 states, 4 actions) value function)

#set up reward so if an action would take it off the board, make it stay still and lose 10 reward, otherwise reward=-1
R = np.array([[-10, -1, -1, -10],
              [-10, -1, -1,  -1],
              [-10,-10, -1,  -1],
              [ -1, -1, -1, -10],
              [ -1, -1, -1,  -1],
              [ -1,-10, -1,  -1],
              [ -1, -1,-10, -10],
              [ -1, -1,-10,  -1],
              [ -1,-10,-10,  -1]])
S0_arr = np.random.randint(0,9,n)  #Generate starting states (1,to 9)

for i in range(n):
    S = S0_arr[i]          #choose random starting state
    Snew = S               #Set Snew to fix the algorithm
    running = True         #true until the terminal state is reached (S = 0 or 8)
    # iteration = 1
    while running:
        if S == 0 or S == 8:
            running = False
        else:
            Amax = get_max_A(Q[S,:])       #Get action that maximizes current move
            Snew = change_state(S,Amax)    #Find out the new state according to the action
            Anewmax = get_max_A(Q[Snew,:]) #Get action that maximizes the next move
            Q[S,Amax] = Q[S,Amax] + alpha*(R[S,Amax]+ gamma*Q[Snew,Anewmax] - Q[S,Amax]) #Update Q
            S = Snew                         #Update state
            # print('Iteration {} at state {}'.format(iteration,S)) #debugging iteration check
            # iteration += 1

print('Final Q: {}'.format(Q))
#Find the optimum policy from Q:
optimum_pi = np.zeros((3,3))
for i in range(3):
    for j in range(3):
        optimum_pi[j,i] = change_state(i+j*3,get_max_A(Q[i+j*3,:]))
optimum_pi[2,2] = 8
print("optimum policy:\n {}".format(optimum_pi))

Final Q: [[ 0.          0.          0.          0.        ]
 [-5.         -1.533125   -1.6090625  -1.        ]
 [-5.         -5.         -1.8998946  -1.89990885]
 [-1.         -1.2        -1.67515625 -5.        ]
 [-1.8975544  -1.89560302 -1.89675097 -1.89659981]
 [-1.67515625 -5.         -1.         -1.65082812]
 [-1.89923634 -1.89937899 -5.         -5.        ]
 [-1.58375    -1.         -5.         -1.18875   ]
 [ 0.          0.          0.          0.        ]]
optimum policy:
 [[0. 0. 5.]
 [0. 5. 8.]
 [3. 8. 8.]]


# More Complicated 4x4 Grid with walls, Q Learning

Using a 4x4 grid with some walls as shown below:

In [6]:
# {F, , , },
# {_,_,_, },
# { , ,_|, },
# { , , , }

Where F is the final state (we only have one final state in the index=0 square this time).
The optimum policy shown has the index in each square of the square that the player should move to. This algorithm converges to an optimum solution, avoiding the walls via a reward incentive (though movement isn't blocked).
Fr_gap defines that every 'fr_gap' iterations, a random value from -10 to -1 is assigned to the reward instead of the correct value, which is discussed in the next section. To remove this effect, set it to 0.

In [20]:
import numpy as np

def grid_4x4_q_learning_ex(alpha,gamma,n,fr_grap):
    #Takes the arr from Q with one state and multiple actions, finds highest value and returns the index of the action column
    def get_max_A(arr):
        indices = np.where(arr == np.amax(arr)) #indices of maximum Q value of the array
        return indices[0][0]

    #takes input of action and returns new state according to board physics
    def change_state(S,A):
        #Don't move if at the board edge. Otherwise 0 up, 1 right, 2 down, 3 left
        if((S % 4 == 0 and A == 3) or (S < 4 and A == 0) or (S > 11 and A == 2) or (S % 4 == 3 and A == 1)):
            return S
        elif(A == 0):
            return S-4
        elif(A == 1):
            return S+1
        elif(A == 2):
            return S+4
        elif(A == 3):
            return S-1
        else:
            print('change_state() broken at state {} and action {}'.format(S,A))
            return S
        
    Q = np.zeros((16,4))                #arbitrary initial Q Table estimate ((9 states, 4 actions) value function)

    #set up reward so if an action would take it off the board, make it stay still and lose 10 reward, otherwise reward
    # {F, , , },
    # {_,_,_, },
    # { , ,_|, },
    # { , , , }
    R = np.array([[-10, -1, -1,-10], [-10, -1, -1, -1], [-10, -1, -1, -1], [-10,-10, -1, -1],
                  [ -1, -1,-10,-10], [ -1, -1,-10, -1], [ -1, -1,-10, -1], [ -1,-10, -1, -1],
                  [-10, -1, -1,-10], [-10, -1, -1, -1], [-10,-10,-10, -1], [ -1,-10, -1,-10],
                  [ -1, -1,-10,-10], [ -1, -1,-10, -1], [-10, -1,-10, -1], [ -1,-10,-10, -1]])
    S0_arr = np.random.randint(0,16,n)  #Generate starting states (1,to 9)

    for i in range(n):
        S = S0_arr[i]          #choose random starting state
        Snew = S               #Set Snew to fix the algorithm
        running = True         #true until the terminal state is reached (S = 0 or 8)
        # iteration = 1
        while running:
            if S == 0:
                running = False
            else:
                Amax = get_max_A(Q[S,:])       #Get action that maximizes current move
                Snew = change_state(S,Amax)    #Find out the new state according to the action
                Anewmax = get_max_A(Q[Snew,:]) #Get action that maximizes the next move
                
                if(fr_grap != 0 and iteration % fr_gap == 0):
                    Q[S,Amax] = Q[S,Amax] + alpha*(rndm.randint(-10,-1) + gamma*Q[Snew,Anewmax] - Q[S,Amax]) #Update Q
                else:
                    Q[S,Amax] = Q[S,Amax] + alpha*(R[S,Amax]+ gamma*Q[Snew,Anewmax] - Q[S,Amax]) #Update Q
                S = Snew                         #Update state
                # print('Iteration {} at state {}'.format(iteration,S)) #debugging iteration check
                # iteration += 1

#     print('Final Q: {}'.format(Q))
    optimum_pi = np.zeros((4,4))
    for i in range(4):
        for j in range(4):
            optimum_pi[j,i] = change_state(i+j*4,get_max_A(Q[i+j*4,:]))
    print("optimum policy:\n {}".format(optimum_pi))
    return optimum_pi

In [26]:
#Call the function with defined parameters
alpha, gamma, n, fr_gap = 0.5, 0.9, 100, 0    #step parameter, discount factor, number of episodes, gap for false rewards
optimum_policy = grid_4x4_q_learning_ex(alpha,gamma,n,fr_gap)

optimum policy:
 [[ 0.  0.  1.  2.]
 [ 0.  1.  2.  6.]
 [12. 13.  9.  7.]
 [13. 14. 15. 11.]]


# Trying to break the 4x4 Q Learning Grid by Feeding False Reward Data

At every 3 steps, it changes the reward recieved to be a random number between -1 and -10 inclusive. I use the same number of iterations (100) as the previous one as that was sufficient to achieve a sensible optimum policy for all attempts, then alpha and gamma are the same. The functions get_max_A() and change_state() are already defined

In [27]:
fr_gap = 3
optimum_policy = grid_4x4_q_learning_ex(alpha,gamma,n,fr_gap)

optimum policy:
 [[ 0.  0.  1.  2.]
 [ 0.  4.  5.  7.]
 [ 4.  5.  9. 10.]
 [ 8. 12. 10. 14.]]


Can see that it definitely confuses a few of the actions and is no longer often the optimum policy. The walls are sometimes broken and the shortest route isn't always taken. If we change to n=1000 we get better routes (shortest routes) but still the walls are broken. Trying it with a larger gap produces less of a noticeable effect as expected, though also many iterations will terminate before having a false reward value inserted in this case.

In [32]:
fr_gap = 5
optimum_policy = grid_4x4_q_learning_ex(alpha,gamma,n,fr_gap)

optimum policy:
 [[ 0.  0.  1.  2.]
 [ 0.  4.  5.  6.]
 [12. 13.  9.  7.]
 [13. 14. 15. 11.]]


Having a fr_gap of only 2 or 1 (1 renders the algorithm fairly useless for avoiding walls as fr_gap=1 would be changing every reward) the results of the algorithm as below.

In [37]:
fr_gap = 2
print("fr_gap is {}".format(fr_gap))
optimum_policy = grid_4x4_q_learning_ex(alpha,gamma,n,fr_gap)
fr_gap = 1
print("fr_gap is {}".format(fr_gap))
optimum_policy = grid_4x4_q_learning_ex(alpha,gamma,n,fr_gap)

fr_gap is 2
optimum policy:
 [[ 0.  0.  1.  2.]
 [ 0.  1.  5.  3.]
 [12. 13.  9.  7.]
 [13. 14. 15. 11.]]
fr_gap is 1
optimum policy:
 [[ 0.  0.  1.  3.]
 [ 0.  4.  5.  3.]
 [ 4.  8. 14. 11.]
 [ 8.  9. 10. 11.]]
